In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import roc_auc_score
import matplotlib as mpl
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import functools
import seaborn as sns
from tqdm import tqdm
from scipy import stats as scpstats
import random

from shapely.geometry import LineString
from matplotlib.ticker import FixedLocator, FixedFormatter,AutoMinorLocator

from scipy import interpolate 
from sklearn.utils import resample

In [ ]:
def df_to_numpy(df):
    df=df.astype(float)
    df=df.T
    df=df.to_numpy()
    return(df)

def FiringRateBis(spiketrain, inf, sup):
    xx = np.arange(inf, sup, 0.001)
    yy = np.zeros_like(xx)
    for spike in spiketrain:
        yy += scpstats.norm.pdf(xx, loc=spike, scale=0.05) #bandwidth de 50 ms
    return(xx,yy)
def format_axes(ax,xlabel,ylabel, reffig, loclegend, pos='right',color='black'):
    ax.text(reffig[0], reffig[1], reffig[2], fontsize=reffig[3], fontweight="bold", transform=ax.transAxes)
    ax.spines[pos].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.set_xlabel(xlabel, fontsize=8)
    ax.set_ylabel(ylabel, fontsize=8, color=color)
    ax.tick_params(axis='x', labelsize=8)
    ax.tick_params(axis='y', labelsize=8)


def intersect2(v, R):    
    
    T1=-extract_flow_velo(v,R)[0]/min(extract_flow_velo(v,R)[0])
    L1=extract_flow_velo(v,R)[1]
    
    l1=np.column_stack((T1[:len(L1.dropna())],L1.dropna()))
    l2=np.column_stack((T1[:len(L1.dropna())],np.repeat(mean_thres,len(l1)) ))

    firstline=LineString(l1)
    secondline = LineString(l2)   
    
    intersectionA = firstline.intersection(secondline)
    vth=intersectionA.xy[0][0]
    pth=100-abs(vth)*100
    
    return(pth)

def return_velo(V,r,interp=False):
    NTIMES=101
    string= "R"+ str(r)+"_v"+str(V)
    T=Vitesses_dico[string] #Rechercher clé dans le dico
    X0=np.linspace(0,6/V,NTIMES) #axs[j].plot(np.linspace(0,6/V,NTIMES),T[:,NPOSES//2],label=str(V))
    X=X0-4/V #mise à zéro
    idx=np.min(np.argwhere(X > 0))
    
    if interp==True:
        tCFD,fCFD=X[:idx+1],T.loc[:idx]
        indices = np.logical_not(np.isnan(np.asarray(fCFD))).flatten()
        tCFD = np.asarray(tCFD)[indices]
        fCFD = np.asarray(fCFD)[indices].squeeze()
        temp = interpolate.interp1d(tCFD, fCFD, fill_value="extrapolate") 
        xnew = np.arange(np.nanmin(X[:idx+1]), 0,0.0001)
        ynew = temp(xnew) 
        return(xnew,ynew)
    
    else:
        return(X[:idx+1],T.loc[:idx] )

def intersect(line1,line2):    
    firstline=LineString(line1)
    secondline = LineString(line2)   
    try:
        intersectionA = firstline.intersection(secondline)
        vth=intersectionA.xy[1][0]
    except:
        vth=np.nan
    return(vth)

def intersect_th(v, R): 
    T1=extract_flow_velo(v,R)[0]
    L1=extract_flow_velo(v,R)[1]
    
    l1=np.column_stack((T1[:len(L1.dropna())],L1.dropna()))
    l2=np.column_stack((T1[:len(L1.dropna())],np.repeat(mean_thres,len(l1)) ))

    firstline=LineString(l1)
    secondline = LineString(l2)   
    
    intersectionA = firstline.intersection(secondline)
    try:
        vth=intersectionA.xy[0][0]
    except:
        vth=np.nan
    return(vth*1000)

In [ ]:
Vitesses_dico ={}
Vitesses_dico["R7.5_v5"]  =pd.read_csv("Volumes/2_Volume_R7.5_V5.csv",header=None)
Vitesses_dico["R7.5_v10"] =pd.read_csv("Volumes/2_Volume_R7.5_V10.csv",header=None)
Vitesses_dico["R7.5_v20"] =pd.read_csv("Volumes/2_Volume_R7.5_V20.csv",header=None)
Vitesses_dico["R7.5_v40"] =pd.read_csv("Volumes/2_Volume_R7.5_V40.csv",header=None)
Vitesses_dico["R15_v5"] =pd.read_csv("Volumes/2_Volume_R15_V5.csv",header=None)
Vitesses_dico["R15_v10"]=pd.read_csv("Volumes/2_Volume_R15_V10.csv",header=None)
Vitesses_dico["R15_v20"]=pd.read_csv("Volumes/2_Volume_R15_V20.csv",header=None)
Vitesses_dico["R15_v40"]=pd.read_csv("Volumes/2_Volume_R15_V40.csv",header=None)
Vitesses_dico["R25_v5"] =pd.read_csv("Volumes/2_Volume_R25_V5.csv",header=None)
Vitesses_dico["R25_v10"]=pd.read_csv("Volumes/2_Volume_R25_V10.csv",header=None)
Vitesses_dico["R25_v20"]=pd.read_csv("Volumes/2_Volume_R25_v20.csv",header=None)
Vitesses_dico["R25_v40"]=pd.read_csv("Volumes/2_Volume_R25_v40.csv",header=None)


df_final = pd.concat(pd.read_excel('Treadmill_clean.xlsx', sheet_name=None), ignore_index=True)
df_final=df_final.loc[(df_final['keep'] == 1) ]
df_final['treac']=df_final['frames_reac']*0.002
df_final_new=df_final
df_final_new['dreac']=df_final_new['v']*df_final_new['treac']-4
df_final_new['treac2'] = df_final_new['treac'] - 4/df_final_new['v']
df_final_new['R_div_v']=df_final_new['R']*0.01/df_final_new['v']

# 1. Extract Behavioural Delays

Outputs:  
- A csv file 
- A pdf file

In [ ]:
class BehavFile:
    def __init__(self,index):
        self.index = index 
        DF=df_final_new
        self.experiment= DF.loc[(DF['idx'] == index)]
        self.tail=self.experiment['idx'].iloc[0]
        self.size=int(self.experiment['R'].iloc[0])/10
        if self.size !=7.5:
            self.size=int(self.size)
    def extract_valves(self, plotting):
        
        self.LINES=[]
        self.LIST_TPEAK=[]
        
        
        self.tab=pd.DataFrame(columns=['file','v','R','treac']) #,'FR_pre'

        t40,f40=return_velo(40,self.size,interp=True)
        t20,f20=return_velo(20,self.size,interp=True)
        t10,f10=return_velo(10,self.size,interp=True)
        t05,f05=return_velo(5, self.size,interp=True)
        simvelo=[f40,f20,f10,f05]
        simtime=[t40,t20,t10,t05]
        velocities=[40,20,10,5]
        colors=['tab:blue','tab:orange','tab:green','tab:red']
        
        stim=np.log2(40/self.experiment['v'])
        stim=[s if np.floor(s)==s else 4.0 for s in stim]


        if plotting:
            fig,ax = plt.subplots(figsize=(6,12))
            ax0=plt.subplot2grid((6,1),(0,0))
            ax5=plt.subplot2grid((6,1),(1,0))
            ax6=plt.subplot2grid((6,1),(2,0))
            ax7=plt.subplot2grid((6,1),(3,0))
            ax8=plt.subplot2grid((6,1),(4,0))
            #ax1B=plt.subplot2grid((6,1),(5,0))
            axscopy=[ax5,ax6,ax7,ax8]

        
        regpts=np.zeros((4,1))
        self.LABELS=[]
        for j in range(4): #0,1,2,3 mais pas 4
            ID=int(stim[j])
            expe_vitesse= self.experiment.iloc[stim.index(ID)]
            
            if plotting: 
                ax0.plot(simtime[ID],simvelo[ID],color=colors[ID])
                axscopy[ID].vlines(expe_vitesse['treac2'],0,np.nanmax(simvelo[ID]), linestyle='dashed',color='grey')
                axscopy[ID].plot(simtime[ID],simvelo[ID],color=colors[ID])

                
            self.LABELS+=[velocities[ID]]    #ordre des vitesses
            line1=np.column_stack((simtime[ID],simvelo[ID]))
            line2=np.column_stack((np.repeat(expe_vitesse['treac2'],len(simvelo[ID])),np.linspace(0,1e-7,len(simvelo[ID]))))
            #vth=intersect(line1,line2)
                    
            self.LINES+=[line1]
            #self.LIST_VTH+=[vth]
            self.LIST_TPEAK+=[expe_vitesse['treac2']]
                    
                    
            new_row = {'file': self.tail, 'R':self.size,'v': 5*2**abs((3-ID)),'treac':expe_vitesse['treac2'] }
            self.tab=pd.concat([self.tab, pd.DataFrame([new_row])], ignore_index=True)

            
def find_threshold_v1(S,ax): #filtrage

    T=pd.DataFrame(columns=['time','m','b','S','pval','R2','adjR','mu','NPTS','TPTS','residuals','keeppoints','var'])

    for delta in np.linspace(0,0.2,500):
        Y=[]
        X=[]
        for k in range(len(S.LINES)):
            l1=S.LINES[k]
            l2=np.column_stack((np.repeat(S.LIST_TPEAK[k]-delta,len(l1)),np.linspace(0,0.401e-7,len(l1))))
            vth=intersect(l1,l2)

            X+=[S.LIST_TPEAK[k]-delta]
            Y+=[vth]
        lentotalpoints=len(X)
        indices = np.logical_not(np.logical_or(np.isnan(X), np.isnan(Y)))
        X = np.array(X)[indices]
        Y = np.array(Y)[indices]

        try:
            X2 = sm.add_constant(X)
            model = sm.OLS(Y, X2)
            results = model.fit()
            influence = results.get_influence()
            standardized_residuals = abs(influence.resid_studentized_internal)
        except:
            standardized_residuals = np.nan

        try:
            X2 = sm.add_constant(X)
            model = sm.OLS(Y, X2)
            results = model.fit()
            influence = results.get_influence()
            standardized_residuals = abs(influence.resid_studentized_internal)
        except:
            standardized_residuals = np.nan

        try:
            b,m=results.params #attention ordre inversé
            pval=results.pvalues[1]
            adjR=results.rsquared_adj
            R2=results.rsquared
        except: 
            b,m=np.nan,np.nan #attention ordre inversé
            pval=np.nan
            adjR=np.nan
            R2=np.nan
        mu=np.mean(Y)
        SUM=0
        for vth in Y:
            SUM+= (vth - mu)**2

        if len(X)>0:
            SUM=np.sqrt(SUM/len(X))      #RMSE
        else:
            SUM=np.nan
            #print('error')

        new_row = {'time':delta,'m':m,'b':b,'S':SUM,'pval':pval,'R2':R2,'adjR':adjR,'mu':mu,'NPTS':len(X),'TPTS':lentotalpoints, 'residuals':standardized_residuals, 'keeppoints': np.sum(standardized_residuals<3), 'var':np.sqrt(np.var(Y))}
        T=pd.concat([T, pd.DataFrame([new_row])], ignore_index=True)

    try:
        TBIS=T.loc[T['NPTS']==max(T['NPTS'])]
        K=T.iloc[abs(TBIS['S']).idxmin()]
    except:
        try:
            K=T.iloc[abs(T['S']).idxmin()]
        except:
            K=T.iloc[-1]
        print('pbm')


    LENGTH=np.arange(0,len(S.LINES),1)
    VTH=[]


    for k,j in zip(LENGTH,S.LABELS):

        if S.size==25:
            color='tab:green'
        elif S.size==15:
            color='tab:orange'
        else:
            color='tab:blue'


        if j==40:
            marker="$40$"
        elif j==20:
            marker="$20$"
        elif j==10:
            marker="$10$"
        else:
            marker="$05$"


        #ax.plot(S.LINES[k][:,0],S.LINES[k][:,1],color='grey')
        ax.plot(S.LINES[k][:,0],S.LINES[k][:,1],alpha=0.2,color=color)
        l1=S.LINES[k]
        l2=np.column_stack((np.repeat(S.LIST_TPEAK[k]-K['time'],len(l1)),np.linspace(0,0.401e-7,len(l1))))
        vth=intersect(l1,l2)
        VTH+=[vth]
        ax.scatter(S.LIST_TPEAK[k]-K['time'],vth,label='v'+str(j)+'_D'+str(S.size),color=color,marker=marker)
        ax.scatter(S.LIST_TPEAK[k]-K['time'],vth,color='grey',alpha=0.1)

    VTH=np.array([v for v in VTH if str(v) != 'nan'])
    ax.hlines(VTH.mean(),-0.8,0,color='grey',linestyle='dashed')
    ax.set_title(str(K['NPTS'])+'/'+str(K['TPTS'])+' pts'+'___'+'RMSE:'+
                 str(np.round(K['S']*1e8,5))+'e-8'+'___'+r'$\delta$ ='+str(np.round(K['time']*1e3,1)))
    ax.set_ylim([0,0.6e-7])
    ax.set_xlabel('time (s)')
    ax.set_ylabel('Linear Momentum')
    ax.legend(loc='best')
    return(T,K)        

In [ ]:
bigtab=pd.DataFrame(columns=['file','v','R','maxFR','timingFR','spikes','stimulus','repet','delta','m','S','pval','R2','adjR','mu','NPTS','TPTS','residuals','keeppoints','var'])

pp = PdfPages('behavioural_delays_test.pdf')

I1=df_final_new.loc[(df_final_new['R'] == 75) & (df_final_new['v'] == 40),'idx']
I2=df_final_new.loc[(df_final_new['R'] == 150) & (df_final_new['v'] == 40),'idx']
I3=df_final_new.loc[(df_final_new['R'] == 250) & (df_final_new['v'] == 40),'idx']


indexes=pd.concat((I1,I2,I3))



for k in range(len(indexes)): #len(valves_files)

    

    S=BehavFile(indexes.iloc[k])
    S.extract_valves(False)


    fig,ax=plt.subplots(figsize=(8,8))
    T1,K=find_threshold_v1(S,ax)
    fig.suptitle(df_final_new.loc[df_final_new['idx']==indexes.iloc[k],'vidéo'].iloc[0][31:31+13])
    plt.show()
    pp.savefig(fig)
    
    S.tab[['delta','m','S','pval','R2','adjR','mu','NPTS','TPTS','keeppoints','var']]=K[['time','m','S','pval','R2','adjR','mu','NPTS','TPTS','keeppoints','var']]
    try:
        res = ','.join(str(x) for x in K['residuals'])
        S.tab['residuals']= res
    except:
        pass
    bigtab=pd.concat([bigtab, S.tab], ignore_index=True)


pp.close()

In [ ]:
# SAVE THE DATA

bigtab2=bigtab.copy()
replacements = {75:7, 150: int(15), 250: int(30)}
bigtab2['R'] = bigtab2['R'].map(replacements).fillna(bigtab2['R'])
bigtab2['R'].astype(int) 

bigtab2.to_csv('delay_behaviour.csv', sep=',') #[['file','v','R','maxFR','timingFR','spikes']]

# 2 Compare delays and momentums

In [ ]:

def data_frame(CLUS):

    df=pd.DataFrame(columns=['D','delay','Mth','S','keep']) #'file',

    I=0
    for t in range(len(CLUS)):
        if t%4==0:

            
   
            signif=1 if CLUS["S"].iloc[t]*1e8<=0.5 and CLUS["TPTS"].iloc[t]==CLUS["NPTS"].iloc[t] else 0    
            if CLUS["delta"].iloc[t]==0.2:
                signif=0
    
                
                
            new_row = {#'file':I, #'$'+CLUS["file"].iloc[t]+'$'
                'D':CLUS["R"].iloc[t],
                           'delay':CLUS["delta"].iloc[t] if CLUS["delta"].iloc[t]!=0 else np.nan ,
                           'Mth': CLUS["mu"].iloc[t],
                       'S':CLUS["S"].iloc[t],
                        'keep': signif
                      }
            df=pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)    

    df=df.rename(columns={'D': "D",
                          'delay': "$\delta^b$",
                       'Mth': "$M_{th}^n$",
                        'S':"$10^{8}*M_{th}$",
                        'keep': "keep"})
    
    #df=df.dropna()
    
    
    print('\\begin{Stable}')
    print(df.to_latex(float_format="%.3e"))
    print('\\caption{Behavioral threshold analysis}')
    print('\\label{tab:beh_vth}')
    print('\\end{Stable}')
    return(df)

def return_color_code(R,v):
    if R==7.5:
        color='tab:blue'
    elif R==15:
        color='tab:orange'
    else:
        color='tab:green'
        
    if v==40:
        style='dashed'
    elif v==20:
        style='solid'
    elif v==10:
        style='dotted'
    else:
        style=(0, (3, 5, 1, 5))
    return(color,style)


## 2.1. Data to Latex Tables

In [ ]:
T0=pd.read_csv('delay_behaviour.csv')
DF=data_frame(T0)
DF

## 2.2 Plots delays and momentums

In [ ]:
def error_bar_delay(DF,ax,color,label,strict, coeff=1):
    
    ax.scatter(coeff*DF.loc[DF["keep"].astype(int)>=0,"$\delta^b$"],DF.loc[DF["keep"].astype(int)>=0,"$10^{8}*M_{th}$"],color='grey',alpha=0.15,marker='x')
    
    if strict==True:
        DATA=DF.loc[DF["keep"].astype(int)>0]
    else:
         DATA=DF.loc[DF["keep"].astype(int)>=0]
            
    ax.scatter(coeff*DATA["$\delta^b$"],DATA["$10^{8}*M_{th}$"], marker='x', color=color)
    #ax.scatter(coeff*DATA["$\delta^n$"],DATA["$10^{8}*M_{th}$"], color=color, alpha=0.2)
    
    mean_delay=coeff*np.nanmean(DATA["$\delta^b$"])
    mean_thres=np.nanmean(DATA["$10^{8}*M_{th}$"])
    error_delay= coeff*np.nanstd(DATA["$\delta^b$"])#/len(DF3["$\delta^n$"])
    error_thres= np.nanstd(DATA["$10^{8}*M_{th}$"])
    ax.scatter(mean_delay, mean_thres, s=30,label=label, facecolors='none', edgecolors=color)
    ax.scatter(mean_delay, mean_thres, s=30, color=color, alpha=0.75)
    ax.hlines(mean_thres, mean_delay-error_delay, mean_delay+error_delay, color=color)
    ax.vlines(mean_delay, mean_thres-error_thres, mean_thres+error_thres, color=color)
    